## Definizioni e caricamenti

### Caricamento dei moduli

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

### Funzioni di aiuto matplotlib

In [ ]:
STD_FIGSIZE = (20, 20)

def to_plot(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

def show_image(image, title, cmap=None, figsize=STD_FIGSIZE):
    figure = plt.figure(figsize=figsize)
    subplot = figure.add_subplot(1, 1, 1)
    subplot.set_title(title)
    subplot.axis('off')
    subplot.imshow(image, cmap=cmap)

def show_side_images(image_1, title_1, image_2, title_2, cmap=None, figsize=STD_FIGSIZE):
    figure = plt.figure(figsize=figsize)
    subplots = figure.subplots(1, 2)
    figure.subplots_adjust(wspace=0.01)
    subplots[0].set_title(title_1)
    subplots[0].axis('off')
    subplots[0].imshow(image_1, cmap=cmap)
    subplots[1].set_title(title_2)
    subplots[1].axis('off')
    subplots[1].imshow(image_2, cmap=cmap)

### Funzioni di aiuto OpenCV

In [ ]:
from cv2 import DMatch

DEF_NFEATURES = 10000
DEF_SIFT_NFEATURES = DEF_NFEATURES
DEF_ORB_NFEATURES = DEF_NFEATURES
DEF_FILTER_DISTANCE = 20
DEF_LOWES_RATIO = 0.75

def convert_to_gray(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

def equalize_image(image):
    return cv2.equalizeHist(image)

def apply_clahe(image):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    output_image = clahe.apply(image)
    return output_image

def blur_image(image, strength=5):
    return cv2.GaussianBlur(image, (strength, strength), 0)

def compute_ecc_transform(image_1, image_2):
    # Initialize the warp matrix
    warp_mode = cv2.MOTION_TRANSLATION
    warp_matrix = np.eye(2, 3, dtype=np.float32)

    # Set the stopping criteria
    criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 5000, 1e-10)

    # Run the ECC algorithm
    cc, warp_matrix = cv2.findTransformECC(image_1, image_2, warp_matrix, warp_mode, criteria)
    return warp_matrix

def apply_ecc_transform(image, warp_matrix, shape=None):
    if shape is None:
        h, w = image.shape[:2]
    else:
        h, w = shape[:2]
    output_image = cv2.warpAffine(image, warp_matrix, (w, h), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)
    return output_image

def filter_matches_with_lowes_ratio(matches: list[DMatch], threshold=DEF_LOWES_RATIO):
    filtered_matches = []
    for m in matches:
        if len(m) == 2 and m[0].distance < threshold * m[1].distance:
            filtered_matches.append(m[0])
    return filtered_matches

def compute_sift_matches(image_1, image_2, nfeatures=DEF_SIFT_NFEATURES, use_lowes_ratio=False):
    sift = cv2.SIFT_create(nfeatures=nfeatures)
    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=not use_lowes_ratio)

    # Detect keypoints and compute descriptors
    keypoints_1, descriptors_1 = sift.detectAndCompute(image_1, None)
    keypoints_2, descriptors_2 = sift.detectAndCompute(image_2, None)

    # Match descriptors
    if use_lowes_ratio:
        matches = bf.knnMatch(descriptors_1, descriptors_2, k=2)
        matches = filter_matches_with_lowes_ratio(matches)
    else:
        matches = bf.match(descriptors_1, descriptors_2)
    matches = sorted(matches, key=lambda x: x.distance)
    return matches, (keypoints_1, keypoints_2)

def compute_orb_matches(image_1, image_2, nfeatures=DEF_ORB_NFEATURES, use_lowes_ratio=False):
    orb = cv2.ORB_create(nfeatures=nfeatures)
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=not use_lowes_ratio)

    # Detect keypoints and compute descriptors
    keypoints_1, descriptors_1 = orb.detectAndCompute(image_1, None)
    keypoints_2, descriptors_2 = orb.detectAndCompute(image_2, None)

    # Match descriptors
    if use_lowes_ratio:
        matches = bf.knnMatch(descriptors_1, descriptors_2, k=2)
        matches = filter_matches_with_lowes_ratio(matches)
    else:
        matches = bf.match(descriptors_1, descriptors_2)
    matches = sorted(matches, key=lambda x: x.distance)
    return matches, (keypoints_1, keypoints_2)

def extract_points(matches: list[DMatch], keypoints: tuple[list, list]):
    points = ([], [])
    for m in matches:
        points[0].append(keypoints[0][m.queryIdx].pt)
        points[1].append(keypoints[1][m.trainIdx].pt)
    return points

def filter_matches_by_euclidean_distance(matches: list[DMatch], keypoints: tuple[list, list], distance=DEF_FILTER_DISTANCE):
    filtered_matches = []
    for m in matches:
        euclidean_distance = np.linalg.norm(np.array(keypoints[0][m.queryIdx].pt) - np.array(keypoints[1][m.trainIdx].pt))
        if euclidean_distance < distance:
            filtered_matches.append(m)
    return filtered_matches

def compute_ransac_transform(points_1, points_2):
    points_1 = np.float32(points_1).reshape(-1,1,2)
    points_2 = np.float32(points_2).reshape(-1,1,2)
    H, mask = cv2.findHomography(points_2, points_1, cv2.RANSAC, 10.0)
    return H

def apply_ransac_transform(image, warp_matrix, shape=None):
    if shape is None:
        h, w = image.shape[:2]
    else:
        h, w = shape[:2]
    output_image = cv2.warpPerspective(image, warp_matrix, (w, h))
    return output_image

def compute_countours(image, threshold1=50, threshold2=150):
    edges = cv2.Canny(image, threshold1=threshold1, threshold2=threshold2)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours

### Apertura delle immagini

In [ ]:
label_free_path = "../Materiale/Prove/non_colorato.png"
stained_path = "../Materiale/Prove/colorato.png"

label_free = cv2.imread(label_free_path)
stained = cv2.imread(stained_path)

label_free_gray = convert_to_gray(label_free)
stained_gray = convert_to_gray(stained)

label_free_equalized = equalize_image(label_free_gray)
stained_equalized = equalize_image(stained_gray)

label_free_clahe = apply_clahe(label_free_gray)
stained_clahe = apply_clahe(stained_gray)

## Immagini iniziali

In [ ]:
show_side_images(to_plot(label_free), "Non colorata", to_plot(stained), "Colorata")
show_side_images(label_free_gray, "Non colorata grigio", stained_gray, "Colorata grigio", cmap='gray')
show_side_images(label_free_equalized, "Non colorata equalizzata", stained_equalized, "Colorata equalizzata", cmap='gray')
show_side_images(label_free_clahe, "Non colorata CLAHE", stained_clahe, "Colorata CLAHE", cmap='gray')

## Allineamento con ECC

In [ ]:
warp_matrix = compute_ecc_transform(label_free_equalized, stained_equalized)
aligned_stained = apply_ecc_transform(stained, warp_matrix, shape=label_free.shape)

aligned_stained_gray = convert_to_gray(aligned_stained)
aligned_stained_equalized = equalize_image(aligned_stained_gray)

difference_og = cv2.absdiff(label_free_equalized, stained_equalized)
difference_aligned = cv2.absdiff(label_free_equalized, aligned_stained_equalized)

show_side_images(to_plot(label_free), "Non colorata", to_plot(aligned_stained), "Colorata allineata")
show_side_images(difference_og, "Differenza assoluta tra immagini originali", difference_aligned, "Differenza assoluta tra immagini allineate", cmap='gray')

## Allinamento tramite Feature Matching con SIFT e immagini originali

In [ ]:
matches, keypoints = compute_sift_matches(label_free, stained)

### Filtraggio dei match

In [ ]:
filtered_matches = filter_matches_by_euclidean_distance(matches, keypoints)

### Display dei match

In [ ]:
drawn_matches = cv2.drawMatches(
    label_free, keypoints[0],
    stained, keypoints[1],
    filtered_matches,
    None,
    flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS,
    matchesThickness=5
)
print(f"Matches: {len(filtered_matches)}")
show_image(to_plot(drawn_matches), "Corrispondenze SIFT")

### Allineamento dell'immagine con RANSAC

In [ ]:
points = extract_points(filtered_matches, keypoints)
warp_matrix = compute_ransac_transform(points[0], points[1])
aligned_stained = apply_ransac_transform(stained, warp_matrix, shape=label_free.shape)
aligned_stained_gray = convert_to_gray(aligned_stained)
aligned_stained_equalized = equalize_image(aligned_stained_gray)
aligned_stained_clahe = apply_clahe(aligned_stained_gray)

difference = cv2.absdiff(label_free_equalized, aligned_stained_equalized)

show_side_images(to_plot(label_free), "Non colorata", to_plot(aligned_stained), "Colorata allineata")
show_side_images(label_free_gray, "Non colorata", aligned_stained_gray, "Colorata allineata", cmap='gray')
show_side_images(label_free_equalized, "Non colorata", aligned_stained_equalized, "Colorata allineata", cmap='gray')
show_side_images(label_free_clahe, "Non colorata", aligned_stained_clahe, "Colorata allineata", cmap='gray')
show_image(difference, "Differenza assoluta tra immagini allineate", cmap='gray')

## Allinamento tramite Feature Matching con SIFT e immagini monocromatiche non equalizzate

In [ ]:
matches, keypoints = compute_sift_matches(label_free_gray, stained_gray)

### Filtraggio dei match

In [ ]:
filtered_matches = filter_matches_by_euclidean_distance(matches, keypoints)

### Display dei match

In [ ]:
drawn_matches = cv2.drawMatches(
    label_free, keypoints[0],
    stained, keypoints[1],
    filtered_matches,
    None,
    flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS,
    matchesThickness=5
)
print(f"Matches: {len(filtered_matches)}")
show_image(to_plot(drawn_matches), "Corrispondenze SIFT")

### Allineamento dell'immagine con RANSAC

In [ ]:
points = extract_points(filtered_matches, keypoints)
warp_matrix = compute_ransac_transform(points[0], points[1])
aligned_stained = apply_ransac_transform(stained, warp_matrix, shape=label_free.shape)
aligned_stained_gray = convert_to_gray(aligned_stained)
aligned_stained_equalized = equalize_image(aligned_stained_gray)
aligned_stained_clahe = apply_clahe(aligned_stained_gray)

difference = cv2.absdiff(label_free_equalized, aligned_stained_equalized)

show_side_images(to_plot(label_free), "Non colorata", to_plot(aligned_stained), "Colorata allineata")
show_side_images(label_free_gray, "Non colorata", aligned_stained_gray, "Colorata allineata", cmap='gray')
show_side_images(label_free_equalized, "Non colorata", aligned_stained_equalized, "Colorata allineata", cmap='gray')
show_side_images(label_free_clahe, "Non colorata", aligned_stained_clahe, "Colorata allineata", cmap='gray')
show_image(difference, "Differenza assoluta tra immagini allineate", cmap='gray')

## Allinamento tramite Feature Matching con SIFT e immagini monocromatiche equalizzate

In [ ]:
matches, keypoints = compute_sift_matches(label_free_equalized, stained_equalized)

### Filtraggio dei match

In [ ]:
filtered_matches = filter_matches_by_euclidean_distance(matches, keypoints)

### Display dei match

In [ ]:
drawn_matches = cv2.drawMatches(
    label_free, keypoints[0],
    stained, keypoints[1],
    filtered_matches,
    None,
    flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS,
    matchesThickness=5
)
print(f"Matches: {len(filtered_matches)}")
show_image(to_plot(drawn_matches), "Corrispondenze SIFT")

### Allineamento dell'immagine con RANSAC

In [ ]:
points = extract_points(filtered_matches, keypoints)
warp_matrix = compute_ransac_transform(points[0], points[1])
aligned_stained = apply_ransac_transform(stained, warp_matrix, shape=label_free.shape)
aligned_stained_gray = convert_to_gray(aligned_stained)
aligned_stained_equalized = equalize_image(aligned_stained_gray)
aligned_stained_clahe = apply_clahe(aligned_stained_gray)

difference = cv2.absdiff(label_free_equalized, aligned_stained_equalized)

show_side_images(to_plot(label_free), "Non colorata", to_plot(aligned_stained), "Colorata allineata")
show_side_images(label_free_gray, "Non colorata", aligned_stained_gray, "Colorata allineata", cmap='gray')
show_side_images(label_free_equalized, "Non colorata", aligned_stained_equalized, "Colorata allineata", cmap='gray')
show_side_images(label_free_clahe, "Non colorata", aligned_stained_clahe, "Colorata allineata", cmap='gray')
show_image(difference, "Differenza assoluta tra immagini allineate", cmap='gray')

## Allinamento tramite Feature Matching con SIFT e immagini monocromatiche CLAHE

In [ ]:
matches, keypoints = compute_sift_matches(label_free_clahe, stained_clahe)

### Filtraggio dei match

In [ ]:
filtered_matches = filter_matches_by_euclidean_distance(matches, keypoints)

### Display dei match

In [ ]:
drawn_matches = cv2.drawMatches(
    label_free, keypoints[0],
    stained, keypoints[1],
    filtered_matches,
    None,
    flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS,
    matchesThickness=5
)
print(f"Matches: {len(filtered_matches)}")
show_image(to_plot(drawn_matches), "Corrispondenze SIFT")

### Allineamento dell'immagine con RANSAC

In [ ]:
points = extract_points(filtered_matches, keypoints)
warp_matrix = compute_ransac_transform(points[0], points[1])
aligned_stained = apply_ransac_transform(stained, warp_matrix, shape=label_free.shape)
aligned_stained_gray = convert_to_gray(aligned_stained)
aligned_stained_equalized = equalize_image(aligned_stained_gray)
aligned_stained_clahe = apply_clahe(aligned_stained_gray)

difference = cv2.absdiff(label_free_equalized, aligned_stained_equalized)

show_side_images(to_plot(label_free), "Non colorata", to_plot(aligned_stained), "Colorata allineata")
show_side_images(label_free_gray, "Non colorata", aligned_stained_gray, "Colorata allineata", cmap='gray')
show_side_images(label_free_equalized, "Non colorata", aligned_stained_equalized, "Colorata allineata", cmap='gray')
show_side_images(label_free_clahe, "Non colorata", aligned_stained_clahe, "Colorata allineata", cmap='gray')
show_image(difference, "Differenza assoluta tra immagini allineate", cmap='gray')

## Cose

### Crea GIF (temporaneo)

In [ ]:
import matplotlib.animation as animation

# Define parameters
duration = 0.5  # seconds
frames = 2
fps = frames / duration

fig, ax = plt.subplots(figsize=STD_FIGSIZE)
ax.axis('off')

# Initialize with fully transparent aligned_stained (alpha=0)
blend_init = cv2.addWeighted(label_free, 1.0, aligned_stained, 0.0, 0)
im = ax.imshow(cv2.cvtColor(blend_init, cv2.COLOR_BGR2RGB))

def update(frame):
    alpha = 0 if frame == 0 else 1
    blended = cv2.addWeighted(label_free, 1 - alpha, aligned_stained, alpha, 0)
    im.set_data(cv2.cvtColor(blended, cv2.COLOR_BGR2RGB))
    return [im]

ani = animation.FuncAnimation(fig, update, frames=frames, interval=1000/fps, blit=True)
ani.save('fade.gif', writer='pillow', fps=fps)
plt.close(fig)

### Contorni

In [ ]:
strength = 1
blur_label_free = blur_image(label_free_gray, strength=strength)
blur_stained = blur_image(stained_gray, strength=strength)
blur_aligned_stained = blur_image(aligned_stained_gray, strength=strength)

# 5. Create a black background
height, width = label_free_gray.shape
contour_mask_blur_label_free = np.zeros((height, width), dtype=np.uint8)
contour_mask_blur_stained = np.zeros((height, width), dtype=np.uint8)
contour_mask_blur_aligned_stained = np.zeros((height, width), dtype=np.uint8)

# 6. Draw contours in white
cv2.drawContours(contour_mask_blur_label_free, compute_countours(blur_label_free), contourIdx=-1, color=255, thickness=1)
cv2.drawContours(contour_mask_blur_stained, compute_countours(blur_stained), contourIdx=-1, color=255, thickness=1)
cv2.drawContours(contour_mask_blur_aligned_stained, compute_countours(blur_aligned_stained), contourIdx=-1, color=255, thickness=1)

intersection_original = cv2.bitwise_and(contour_mask_blur_label_free, contour_mask_blur_stained)
intersection_aligned = cv2.bitwise_and(contour_mask_blur_label_free, contour_mask_blur_aligned_stained)

show_side_images(label_free_gray, "Non colorata", aligned_stained_gray, "Colorata", cmap='gray')
show_side_images(contour_mask_blur_label_free, "Contorni non colorata", contour_mask_blur_stained, "Contorni colorata", cmap='gray')
show_side_images(contour_mask_blur_label_free, "Contorni non colorata", contour_mask_blur_aligned_stained, "Contorni colorata allineata", cmap='gray')
show_side_images(intersection_original, "Intersezione contorni originali", intersection_aligned, "Intersezione contorni allineati", cmap='gray')

zoomed_countour_mask_1 = cv2.resize(label_free_gray, (0, 0), fx=2, fy=2)[800:1600, 800:1600]
zoomed_countour_mask_2 = cv2.resize(aligned_stained_gray, (0, 0), fx=2, fy=2)[800:1600, 800:1600]

### Metriche

In [ ]:
from skimage.metrics import structural_similarity as ssim

blur_label_free = cv2.GaussianBlur(label_free_gray, (5, 5), 0)
blur_stained = cv2.GaussianBlur(stained_gray, (5, 5), 0)
blur_aligned_stained = cv2.GaussianBlur(aligned_stained_gray, (5, 5), 0)

score, diff_map = ssim(contour_mask_blur_label_free, contour_mask_blur_aligned_stained, full=True)
print("SSIM:", score)

def mutual_information(image1, image2, bins=256):
    """
    image1, image2: single-channel images (e.g., grayscale, same shape)
    bins: number of intensity bins for the histogram
    """
    # Flatten the images to 1D
    i1 = image1.flatten()
    i2 = image2.flatten()
    
    # Compute 2D joint histogram
    joint_hist, x_edges, y_edges = np.histogram2d(i1, i2, bins=bins)
    
    # Convert to joint probability by dividing by total number of pixels
    joint_prob = joint_hist / np.sum(joint_hist)
    
    # Marginal probabilities
    p1 = np.sum(joint_prob, axis=1)  # sum over columns -> distribution of image1
    p2 = np.sum(joint_prob, axis=0)  # sum over rows -> distribution of image2

    # Avoid log(0) by adding small epsilon
    eps = 1e-10
    # MI formula
    mi = 0.0
    for i in range(bins):
        for j in range(bins):
            p_ij = joint_prob[i, j]
            if p_ij > eps:
                mi += p_ij * np.log(p_ij / (p1[i]*p2[j] + eps) + eps)

    return mi

mi_value = mutual_information(label_free, aligned_stained)
print("Mutual Information:", mi_value)
mi_value = mutual_information(label_free_gray, aligned_stained_gray)
print("Mutual Information:", mi_value)
mi_value = mutual_information(label_free_clahe, aligned_stained_clahe)
print("Mutual Information:", mi_value)
mi_value = mutual_information(blur_label_free, blur_stained)
print("Mutual Information:", mi_value)
mi_value = mutual_information(blur_label_free, blur_aligned_stained)
print("Mutual Information:", mi_value)
mi_value = mutual_information(contour_mask_blur_label_free, contour_mask_blur_aligned_stained)
print("Mutual Information:", mi_value)

## Allinamento molteplico tramite Feature Matching con SIFT e immagini monocromatiche equalizzate

In [ ]:
aligned_stained = stained
aligned_stained_equalized = stained_equalized

N = 1

for i in range(N):
    matches, keypoints = compute_sift_matches(label_free_equalized, aligned_stained_equalized)
    filtered_matches = filter_matches_by_euclidean_distance(matches, keypoints)

    drawn_matches = cv2.drawMatches(
        label_free, keypoints[0],
        aligned_stained, keypoints[1],
        filtered_matches,
        None,
        flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS,
        matchesThickness=5
    )

    print(f"Matches: {len(filtered_matches)}")
    #show_image(to_plot(drawn_matches), "Corrispondenze SIFT")

    points = extract_points(filtered_matches, keypoints)
    warp_matrix = compute_ransac_transform(points[0], points[1])
    print(i, warp_matrix)
    aligned_stained = apply_ransac_transform(aligned_stained, warp_matrix, shape=label_free.shape)
    aligned_stained_gray = convert_to_gray(aligned_stained)
    aligned_stained_equalized = equalize_image(aligned_stained_gray)

difference = cv2.absdiff(label_free_gray, aligned_stained_gray)

show_side_images(to_plot(label_free), "Non colorata", to_plot(aligned_stained), "Colorata allineata")
show_side_images(label_free_gray, "Non colorata", aligned_stained_gray, "Colorata allineata", cmap='gray')
show_side_images(label_free_equalized, "Non colorata", aligned_stained_equalized, "Colorata allineata", cmap='gray')
show_image(difference, "Differenza assoluta tra immagini allineate", cmap='gray')

## Controllo con metriche

In [ ]:
def mutual_information(img1: np.ndarray, img2: np.ndarray, bins: int = 256) -> float:
    """
    Compute the mutual information (MI) between two grayscale images.
    
    :param img1: First image in grayscale.
    :param img2: Second image in grayscale.
    :param bins: Number of histogram bins to use (256 for 8-bit images).
    :return: Mutual information (in bits) between img1 and img2.
    """
    # Convert images to 1D numpy arrays (flatten) if they are not already
    img1 = img1.ravel()
    img2 = img2.ravel()

    # Compute the joint histogram
    # histogram2d returns the 2D histogram and bin edges
    joint_hist, x_edges, y_edges = np.histogram2d(img1, img2, bins=bins)
    
    # Normalize the joint histogram to get the joint PDF p(x,y)
    joint_pdf = joint_hist / np.sum(joint_hist)
    
    # Compute the marginal PDFs p(x) and p(y)
    p_x = np.sum(joint_pdf, axis=1)  # Sum over columns -> marginal over x
    p_y = np.sum(joint_pdf, axis=0)  # Sum over rows -> marginal over y

    # Only consider non-zero values to avoid log(0)
    # MI = sum p(x,y)*log( p(x,y)/(p(x)*p(y)) )
    non_zero_idxs = joint_pdf > 0
    mi = np.sum(joint_pdf[non_zero_idxs] * 
                np.log2(joint_pdf[non_zero_idxs] / 
                        (p_x[np.newaxis].T @ p_y[np.newaxis])[non_zero_idxs]))
    
    return mi

MODE_SIFT = 0
MODE_ORB = 1

def align(image_1, image_2, image_to_align=None, image_to_confront=None, mode=MODE_SIFT, use_lowes_ratio=False):
    if image_to_align is None:
        image_to_align = image_2
    if image_to_confront is None:
        image_to_confront = image_1
    if mode == MODE_SIFT:
        matches, keypoints = compute_sift_matches(image_1, image_2, use_lowes_ratio=use_lowes_ratio)
        print(f"Matches: {len(matches)}", end=" - ")
    elif mode == MODE_ORB:
        matches, keypoints = compute_orb_matches(image_1, image_2, use_lowes_ratio=use_lowes_ratio)
    else:
        raise Exception("Mode not supported")
    filtered_matches = filter_matches_by_euclidean_distance(matches, keypoints)
    points = extract_points(filtered_matches, keypoints)
    warp_matrix = compute_ransac_transform(points[0], points[1])
    aligned_image = apply_ransac_transform(image_to_align, warp_matrix, shape=image_1.shape)
    blurred_aligned_image = blur_image(aligned_image, strength=1)
    mi = mutual_information(image_to_confront, blurred_aligned_image)
    return aligned_image, mi

def test_all_colors(mode=MODE_SIFT, use_lowes_ratio=False):
    mode = "SIFT" if mode == MODE_SIFT else "ORB"
    _, mi = align(label_free_clahe, label_free_clahe, image_to_align=label_free_gray, image_to_confront=label_free_gray)
    print(f"{mode} - LR: {'T' if use_lowes_ratio else 'F'} - SAME - MI: {mi}")
    _, mi = align(label_free, stained, image_to_align=stained_gray, image_to_confront=label_free_gray)
    print(f"{mode} - LR: {'T' if use_lowes_ratio else 'F'} - ORIGINAL - MI: {mi}")
    _, mi = align(label_free_gray, stained_gray, image_to_align=stained_gray, image_to_confront=label_free_gray)
    print(f"{mode} - LR: {'T' if use_lowes_ratio else 'F'} - GRAY - MI: {mi}")
    _, mi = align(label_free_equalized, stained_equalized, image_to_align=stained_gray, image_to_confront=label_free_gray)
    print(f"{mode} - LR: {'T' if use_lowes_ratio else 'F'} - EQUALIZED - MI: {mi}")
    _, mi = align(label_free_clahe, stained_clahe, image_to_align=stained_gray, image_to_confront=label_free_gray)
    print(f"{mode} - LR: {'T' if use_lowes_ratio else 'F'} - CLAHE - MI: {mi}")

test_all_colors(mode=MODE_SIFT, use_lowes_ratio=False)
test_all_colors(mode=MODE_SIFT, use_lowes_ratio=True)
test_all_colors(mode=MODE_ORB, use_lowes_ratio=False)
test_all_colors(mode=MODE_ORB, use_lowes_ratio=True)